In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import nltk.data
from nltk.tokenize import BlanklineTokenizer
import sys
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
test = open("/content/drive/MyDrive/Hybridization/input/en-hi-wsd-3L/output_en-hi_WSD-500_intrain.txt", "r")
reference = open("/content/drive/MyDrive/Hybridization/output/en-hi/ref_en-hi-500-intrain.txt", "r")

In [ ]:
list1 = []
list2 = []
for line in reference:
    #print(line)
    list1.append(BlanklineTokenizer().tokenize(line))
for line in test:
    #print(line)
    list2.append(BlanklineTokenizer().tokenize(line))
reference.close()
test.close()

In [ ]:
s1=[]
s2=[]
for ele in list1:
    for elem in ele:
        s1.append(" ".join([str(elem) for elem in ele]))
for ele in list2:
    for elem in ele:
        s2.append(" ".join([str(elem) for elem in ele]))

In [ ]:
le=len(s2)
te = len(s1)
print(le)
print(te)

500
500


In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet
from itertools import chain, product


def _generate_enums(hypothesis, reference, preprocess=str.lower):
    hypothesis_list = list(enumerate(preprocess(hypothesis).split()))
    reference_list = list(enumerate(preprocess(reference).split()))
    return hypothesis_list, reference_list

def _enum_stem_match(enum_hypothesis_list, enum_reference_list, stemmer = PorterStemmer()):
    stemmed_enum_list1 = [(word_pair[0],stemmer.stem(word_pair[1])) \
                          for word_pair in enum_hypothesis_list]

    stemmed_enum_list2 = [(word_pair[0],stemmer.stem(word_pair[1])) \
                          for word_pair in enum_reference_list]

    word_match, enum_unmat_hypo_list, enum_unmat_ref_list = \
                    _match_enums(stemmed_enum_list1, stemmed_enum_list2)

    enum_unmat_hypo_list = list(zip(*enum_unmat_hypo_list)) if len(enum_unmat_hypo_list)>0 else []

    enum_unmat_ref_list = list(zip(*enum_unmat_ref_list)) if len(enum_unmat_ref_list)>0 else []

    enum_hypothesis_list = list(filter(lambda x:x[0] not in enum_unmat_hypo_list,  
                                       enum_hypothesis_list))

    enum_reference_list = list(filter(lambda x:x[0] not in enum_unmat_ref_list, 
                                      enum_reference_list))

    return word_match, enum_hypothesis_list, enum_reference_list




def _match_enums(enum_hypothesis_list, enum_reference_list):
    word_match = []
    for i in range(len(enum_hypothesis_list))[::-1]:
        for j in range(len(enum_reference_list))[::-1]:
            if enum_hypothesis_list[i][1] == enum_reference_list[j][1]:
                word_match.append((enum_hypothesis_list[i][0],enum_reference_list[j][0]))
                (enum_hypothesis_list.pop(i)[1],enum_reference_list.pop(j)[1])
                break
    return word_match, enum_hypothesis_list, enum_reference_list


def _enum_wordnetsyn_match(enum_hypothesis_list, enum_reference_list, wordnet = wordnet):
    word_match = []
    for i in range(len(enum_hypothesis_list))[::-1]:
        hypothesis_syns = set(chain(*[[lemma.name() for lemma in synset.lemmas() 
                                        if lemma.name().find('_')<0] 
                                       for synset in \
                                           wordnet.synsets(
                                               enum_hypothesis_list[i][1])]
                                               )).union({enum_hypothesis_list[i][1]})
        for j in range(len(enum_reference_list))[::-1]:
            if enum_reference_list[j][1] in hypothesis_syns:
                word_match.append((enum_hypothesis_list[i][0],enum_reference_list[j][0]))
                enum_hypothesis_list.pop(i),enum_reference_list.pop(j)
                break
    return word_match, enum_hypothesis_list, enum_reference_list


def _count_chunks(matches):
    i=0
    chunks = 1
    while(i<len(matches)-1):
        if (matches[i+1][0]==matches[i][0]+1) and (matches[i+1][1]==matches[i][1]+1):
            i+=1
            continue
        i+=1
        chunks += 1
    return chunks






def _enum_allign_words(enum_hypothesis_list, enum_reference_list, 
                       stemmer=PorterStemmer(), wordnet = wordnet):
    exact_matches, enum_hypothesis_list, enum_reference_list = \
        _match_enums(enum_hypothesis_list, enum_reference_list)

    stem_matches, enum_hypothesis_list, enum_reference_list = \
        _enum_stem_match(enum_hypothesis_list, enum_reference_list,
                         stemmer = stemmer)

    wns_matches, enum_hypothesis_list, enum_reference_list = \
        _enum_wordnetsyn_match(enum_hypothesis_list, enum_reference_list, 
                               wordnet = wordnet)

    return (sorted(exact_matches + stem_matches + wns_matches, key=lambda wordpair:wordpair[0]),
            enum_hypothesis_list, enum_reference_list)




def single_meteor_score(reference, 
                        hypothesis, 
                        preprocess = str.lower, 
                        stemmer = PorterStemmer(), 
                        wordnet = wordnet, 
                        alpha=0.9, 
                        beta=3, 
                        gamma=0.5):
    enum_hypothesis, enum_reference = _generate_enums(hypothesis, 
                                                      reference, 
                                                      preprocess = preprocess)
    #print(enum_hypothesis, enum_reference)
    
    translation_length = len(enum_hypothesis)
    reference_length = len(enum_reference)
    matches, _, _ = _enum_allign_words(enum_hypothesis, enum_reference)
    
    #print(matches, _, _)
    matches_count = len(matches)
    try:
        precision = float(matches_count)/translation_length
        recall = float(matches_count)/reference_length
        fmean = (precision*recall)/(alpha*precision+(1-alpha)*recall)
        chunk_count = float(_count_chunks(matches))
        frag_frac = chunk_count/matches_count
    except ZeroDivisionError:
        return 0.0        
    penalty = gamma*frag_frac**beta
    return (1-penalty)*fmean


In [ ]:
lop=0
scor=[]
coun=0
for lop in range(le):
    scor.append(single_meteor_score(s1[lop],s2[lop]))

In [ ]:
f1 = open('/content/meteor_blank_score_ur1.txt', "w")
lope=0
su=0
for lope in range(le):
    print(f"{lope+1}.   {scor[lope]}")
    su=su+scor[lope]
    f1.write(str(scor[lope])+'\n') #Sentence_score
f1.close()
su=su/le
su #Corpus_score

1.   0.0
2.   0.05649717514124294
3.   0.0
4.   0.03496503496503497
5.   0.0
6.   0.0
7.   0.04178272980501392
8.   0.0
9.   0.0
10.   0.08986928104575162
11.   0.0
12.   0.0
13.   0.03906250000000001
14.   0.02040816326530612
15.   0.0
16.   0.09920634920634921
17.   0.01779359430604982
18.   0.0
19.   0.03906250000000001
20.   0.019157088122605363
21.   0.0
22.   0.019230769230769232
23.   0.0
24.   0.0
25.   0.06930693069306931
26.   0.0
27.   0.02100840336134454
28.   0.029411764705882353
29.   0.060728744939271245
30.   0.05555555555555555
31.   0.0
32.   0.03571428571428571
33.   0.0
34.   0.04347826086956522
35.   0.07142857142857142
36.   0.07936507936507936
37.   0.05405405405405404
38.   0.0
39.   0.0
40.   0.05190311418685121
41.   0.07042253521126761
42.   0.19736842105263158
43.   0.029411764705882353
44.   0.02590673575129534
45.   0.025252525252525256
46.   0.0
47.   0.03355704697986577
48.   0.04807692307692309
49.   0.06097560975609756
50.   0.022883295194508005
51.   

0.030915588366500376

In [ ]:
f1 = open('/content/meteor_blank_score_ur1.txt', "a+")
with open(translated_file, "r") as f1, open(reference_file, "r") as f2:
  lines1 = [line.rstrip('\n') for line in f1]
  print(len(lines1))
  lines2 = [line.rstrip('\n') for line in f2]
  print(len(lines2))
  for l1,l2 in zip(lines1,lines2):
    tokens_ref=[word_tokenize(l2)]
    #print(tokens_ref)
    tokens_translate=word_tokenize(l1)
    #print(tokens_translate)
    score = nltk.translate.nist_score.sentence_nist(tokens_ref, tokens_translate,n=4)
    f5.write(str(score)+'\n')
f5.close()

NameError: ignored